In [ ]:
import pandas as pd
import sqlalchemy

%load_ext sql

# Set a few config options to prettify the output and return it as Pandas DataFrame
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:
# DuckDB is primarily designed to be an in-memory DB. You can however persist your data to disk.
# %sql duckdb:///path/to/file.db

In [ ]:
%%sql
SELECT 1 AS a;

In [ ]:
%%sql

SELECT *
FROM './csv-data/2009.csv'
LIMIT 5;

In [ ]:
pd.read_csv('./csv-data/2009.csv').head(5)

In [ ]:
pd.read_csv('./csv-data/2009.csv', nrows=5)

In [ ]:
%%sql

-- copy from csv to parquet, this takes 6 minutes

DROP TABLE IF EXISTS TempTable;

CREATE TABLE TempTable
AS SELECT * FROM './csv-data/*.csv';

ALTER TABLE TempTable
DROP COLUMN "Unnamed: 27";

COPY TempTable TO 'parquet-data/data.parquet';

In [ ]:
! echo "csv data" && ls -lh csv-data/
! echo "parquet data" && ls -lh parquet-data/data.parquet

In [ ]:
%%sql

SELECT OP_CARRIER, MAX(ARR_DELAY) AS max_delay
FROM 'parquet-data/data.parquet'
GROUP BY OP_CARRIER
ORDER BY max_delay DESC
LIMIT 5;

In [ ]:
import duckdb

top_avg_5 = duckdb.query(
    """
    SELECT OP_CARRIER, AVG(ARR_DELAY) AS avg_delay
    FROM 'parquet-data/data.parquet'
    GROUP BY OP_CARRIER
    ORDER BY avg_delay DESC
    LIMIT 5;
    """
).to_df()

In [ ]:
top_avg_5.plot.bar(x="OP_CARRIER");

In [ ]:
%%sql

CREATE VIEW airlinedata AS SELECT * FROM 'parquet-data/data.parquet';

SELECT OP_CARRIER, MIN(ARR_DELAY), AVG(ARR_DELAY), MAX(ARR_DELAY)
FROM airlinedata
GROUP BY OP_CARRIER;